In [1]:
from __future__ import print_function
import argparse
import os
import random

import numpy

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.utils import np_utils
import keras.models as models
from keras.layers import Input,merge
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import *
from keras.layers.wrappers import TimeDistributed
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, UpSampling2D,Conv2DTranspose
from keras.layers.recurrent import LSTM
from keras.regularizers import *
from keras.layers.normalization import *
from keras.optimizers import *

Using TensorFlow backend.


In [2]:
dataset='cifar10'
dataroot='f1'
workers=1
batchSize=5
imageSize=32
nz=400
ngf=64
ndf=64
niter=25
lr=0.0002
beta1=0.5
cuda=True
ngpu=1
netG=''
netD=''
outf='f1'
manualSeed=10
try:
    os.makedirs(outf)
except OSError:
    pass


In [3]:
import os
I='A/train/'
ls1=os.listdir(I)
M='B/train/'
ls2=os.listdir(M)[0:100]

import scipy.misc as sm
import numpy as np
import scipy.io as si
names=[]
lls2=len(ls2)
m=int(0.9*lls2)

X=np.zeros((m,128,128,3))
Y=np.zeros((m,256,256,3))
Xtest=np.zeros((lls2-m,128,128,3))
Ytest=np.zeros((lls2-m,256,256,3))
count=0
for ind,i in enumerate(ls2):
    print(ind)
    if(ind<m):
        names.append(i)
        #temp=i.split('__')[0]
        tmp1=sm.imread(I+i)
        X[ind,:]=tmp1/255.0
        tmp1=sm.imread(M+i)
        Y[ind,:]=tmp1/255.0
    else:
        #break
        names.append(i)
        tmp1=sm.imread(I+i)
        Xtest[count,:]=tmp1/255.0
        tmp1=sm.imread(M+i)
        Ytest[count,:]=tmp1/255.0
        count=count+1
    
    
    
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [26]:
def larger_model():
    model = Sequential()
    model.add(Convolution2D(32,(4,4),activation='relu',strides=2,input_shape=(128,128,3) ))
    model.add(Convolution2D(32*2,(4,4),activation='relu',strides=2 ))
    model.add(Convolution2D(32*4,(4,4),activation='relu',strides=2))
    model.add(Conv2DTranspose(32*4,(4,4),activation='relu',strides=2))
    model.add(Conv2DTranspose(32*4,(4,4),activation='relu',strides=2))
    model.add(Conv2DTranspose(3,(4,4),activation='relu',strides=2))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2DTranspose(3,(2,2),activation='relu',strides=2))
    return model


In [27]:
def discr():
    model = Sequential()
    model.add(Convolution2D(32,(4,4),activation='relu',strides=2,input_shape=(256,256,3) ))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(32*2,(4,4),activation='relu',strides=2 ))
    model.add(Convolution2D(32*4,(4,4),activation='relu',strides=2))
    model.add(Convolution2D(32*2,(4,4),activation='relu',strides=2))
    model.add(Convolution2D(1,(4,4),activation='relu',strides=2))
    
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model


In [28]:
def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val
        
gen=larger_model()
dis=discr()
make_trainable(dis, False)

In [29]:
def mixed(x,y):
    model=Sequential()
    model.add(x)
    model.add(y)
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model
    

In [30]:
GAN=mixed(gen,dis)
GAN.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_11 (Sequential)   (None, 256, 256, 3)       696330    
_________________________________________________________________
sequential_12 (Sequential)   (None, 6, 6, 1)           297761    
Total params: 994,091
Trainable params: 696,330
Non-trainable params: 297,761
_________________________________________________________________


In [31]:
bs=20
make_trainable(dis,True)

for epoch in range(100):
    for i in range(100):
        ind=np.random.permutation(X.shape[0])[0:bs]
        data=X[(ind)]        
        data2=Y[(ind)]

        gen_im=gen.predict(data)
        train_D = np.concatenate((data2, gen_im))
        
        y_D=np.zeros((2*bs,6,6,1))
        y_D[0:bs,:]=1
        y_D[bs:,:]=0
        d_loss  = dis.train_on_batch(train_D,y_D)
        #print(gen_im.shape)

        y_G=np.zeros((bs,6,6,1))
        y_G[:,:]=1
        g_loss=GAN.train_on_batch(data,y_G)
        if(i%40==0):
            print((epoch,i,d_loss,g_loss))
        #break
    #break

(0, 0, 6.8470807, 4.6162291)
(0, 40, 7.9711947, 1.1920935e-07)


KeyboardInterrupt: 

In [32]:
import scipy.misc as sm
temp=gen.predict(Xtest[50:51,:])
print(temp)
sm.imsave('vap.png',temp.reshape((256,256,3)))
sm.imsave('vap2.png',Ytest[50:51].reshape((256,256,3)))


[]


AttributeError: 'list' object has no attribute 'reshape'